In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

C:\Users\Jorrit\Miniconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Jorrit\Miniconda3\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
C:\Users\Jorrit\Miniconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
torch.manual_seed(4242)

In [10]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data/p1ch2/mnist', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=64, shuffle=True)

Using downloaded and verified file: ../data/p1ch2/mnist\MNIST\raw\train-images-idx3-ubyte.gz
Extracting ../data/p1ch2/mnist\MNIST\raw\train-images-idx3-ubyte.gz to ../data/p1ch2/mnist\MNIST\raw




Extracting ../data/p1ch2/mnist\MNIST\raw\train-labels-idx1-ubyte.gz to ../data/p1ch2/mnist\MNIST\raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ../data/p1ch2/mnist\MNIST\raw\t10k-images-idx3-ubyte.gz to ../data/p1ch2/mnist\MNIST\raw




Extracting ../data/p1ch2/mnist\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data/p1ch2/mnist\MNIST\raw



C:\Users\Jorrit\Miniconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [12]:
model = Net()

In [13]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [14]:
for epoch in range(10):
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
    print('Current loss', float(loss))

C:\Users\Jorrit\Miniconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Current loss 0.37703508138656616
Current loss 0.13915598392486572
Current loss 0.10450372844934464
Current loss 0.1123143807053566
Current loss 0.05952317640185356
Current loss 0.16038605570793152
Current loss 0.16485361754894257
Current loss 0.03149827942252159
Current loss 0.07533768564462662
Current loss 0.11868184804916382


In [15]:
torch.save(model.state_dict(), '../data/p1ch2/mnist/mnist.pth')

In [16]:
pretrained_model = Net()
pretrained_model.load_state_dict(torch.load('../data/p1ch2/mnist/mnist.pth'))

<All keys matched successfully>